# Remote Sensing Exercise - Part B

In this  exercise you will analyze some data from our SUHI database.

In particular, what we will do:

1. Open the Database and use SQL to retrieve data from it.
2. Calculate the daily SUHI intensity (SUHII) for Mexico city.
3. Caclulate the monthly and annual means for SUHII and rural LST.
4. Visualize the SUHII seasonal hysteresis of Mexico city.
5. Visualize the SUHII seasonal hystereis in each densely-populated climate of Earth.

## Database Tables

The SUHI database includes six tables that store the aggregated satellite data (2000-2020) and other ancillary information. These six tables are the following:

| Table        | Description                                               | 
| ------------ | ----------------------------------------------------------| 
| `cities`     | The cities and their atrributes.                          | 
| `countries`  | The world's countries and their chracteristics.           |
| `data_day`   | The daytime data.                                         |
| `data_night` | The nighttime data.                                       |
| `city_size`  | The area  [km<sup>2</sup>] of each city polygon per year. |
| `land_covers`| The land cover legend.                                    |

To view the database in your pc you can use this software: https://dbeaver.io/.

******
**Copyright: 2022, Panagiotis Sismanidis**
******


## Setting up what we need...

In [ ]:
# First load your Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# After mounting, check folders in left side-bar
# If this code does not work, you can use the "Mount Drive" button in the left side-bar

In [ ]:
# Set relevant directories
import os

# Your own baseline directory - if mounted under MyDrive, do not change.
BASE_DIR = os.path.join('/content/drive/MyDrive/BUCSS22')
print('BASE_DIR: ',BASE_DIR)

# Create a link to notebooks directory
YOUR_NAME = "Panagiotis_Lecturer" # Name of your own folder
NOTEBOOKS_DIR = os.path.join(BASE_DIR, YOUR_NAME, 'notebooks')
print('NOTEBOOKS_DIR: ',NOTEBOOKS_DIR)

# Other relevant folders
LST_DIR         = os.path.join(BASE_DIR, 'DATA_SHARE', 'LST')
print('DATA_DIR: ',LST_DIR)

# Basics

To retrieve data from an SQLite database we will use the [`pandas.read_sql()`](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html) method, which will return a `pandas.Dataframe` with the queried data. This method takes as input a string describing a SQL `SELECT` statement. The `SELECT` statement is used to select data from a database and in its simplest form it looks like this:
```sql
SELECT column1, column2, ...
FROM table_name;
```
where, `column1`, and `column2` are the field names of the table you want to select data from and `table_name` is the name of the table.

## B.1 Simplest case

**Goal**: Retrieve all the rows from the columns `UID` (used instead of name), `country`, `latitude`, and `longitude` in the table `cities`.

In [ ]:
# Step 1: import the `pandas` and `sqlite3` modules
import pandas as pd
import sqlite3

# Step 2: open a connection to the database.
filename = os.path.join(LST_DIR, "SUHI_DB-MOD11A1.061-v1.0.1.db")
connection = sqlite3.connect(filename)

# Step 3: create a sql query
sql_query = "SELECT UID, country, latitude, longitude FROM cities"

# Step 4: retieve data from the database
cities = pd.read_sql(
    sql = sql_query,
    con = connection,  
)

# Step 5: close the database connection
connection.close()

# and you are done! :)
cities.head()

To check the number of rows, use the `len()` method:

In [ ]:
print(f"Number of rows: {len(cities)}")

### B.2 Select Rows based on Condition

**Goal**: Same as above, but now keep only the rows where the city elevation is at least 200 m and the climate is classified as oceanic.

In [ ]:
import pandas as pd
import sqlite3

filename = os.path.join(LST_DIR, "SUHI_DB-MOD11A1.061-v1.0.1.db")
connection = sqlite3.connect(filename)

# select only the cities where the climate is oceanic (Cfb)
# and the city elevation is equal or greater than 200 m.
sql_query ="""
    SELECT UID, country, latitude, longitude
    FROM cities
    WHERE climate = "Cwb"
    AND elevation >= 200
    """

cities = pd.read_sql(
    sql = sql_query,
    con = connection,  
)

connection.close()

print(cities.head())
print(f"Number of rows: {len(cities)}")

Now, let's do the same without hardcoding the threshold values:

In [ ]:
import pandas as pd
import sqlite3

filename = os.path.join(LST_DIR, "SUHI_DB-MOD11A1.061-v1.0.1.db")
connection = sqlite3.connect(filename)

sql_query =\
"""
    SELECT UID, country, latitude, longitude, elevation
    FROM cities
    WHERE climate=:climate_zone
    AND elevation>=:min_elevation
"""

# Create a dictionary to store the threshold values.
# The keys should have the same names as in the sql_query!
query_args = {
    "climate_zone": "Cwb",
    "min_elevation": 200,
}

cities = pd.read_sql(
    sql = sql_query,
    con = connection,
    params = query_args,
)

connection.close()

display(cities.head())
print(f"Number of rows: {len(cities)}")

## Let's get started...

In [ ]:
# First let's import all the modules we need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
%matplotlib inline

plt.rcParams['figure.dpi'] = 100
sns.set_style("darkgrid")
sns.set_context("notebook")

We will start by creating a function to get data from the database.

The function should take as input arguments the city ID and the land cover, where:

| Land Cover | Value |
| ---------- | ----- |
| Urban      | 0 |
| Rural      | 1 |

Let's use the function to get the LST data for the City of Mexico (`UID = "MEX-19957"`).

The number of rows is equal to:

> **Task**
> 
> Try to change the default values for `vza_median` and `pxl_perc` values in `sql_query` and see how it affects the dataframe size.

Before we move on, let's plot the LST means for Mexico City.

In [ ]:
# plot the time series


## Task 2: Calculate the SUHI Intensity

To calculate the SUHI intensity we also need the rural LST.

In [ ]:
# Let's visualize the two datasets.


To calculate the SUHI intensity we first need to join the two dataframes:

In [ ]:
# Use the merge() method to join the two dataframes.


Calculate the urban and rural LST differences per day and store it into a new column:

In [ ]:
# summarize our results


In [ ]:
# Let's also plot how SUHII varies temporally


## Task 3: Calculate the Annual and Monthly Means

In [ ]:
# We will start with the annual means


In [ ]:
# and continue with the monhtly means


### Task 4: Visualize the SUHII Hysteresis

Let's try to visualize the time lag between rural LST and SUHI intensity for Mexico City.

In [ ]:
# sort them according to month
monthly_means.sort_values(by=["month"], inplace=True)

# close the loop
means = monthly_means.append(monthly_means.iloc[0]) 

# Create a figure
fig, ax = plt.subplots()

sns.histplot(
    data=df_mexcity,
    x="lst_mean_rural",
    y="suhii",
    binwidth=[1, 0.5],
    cmap="crest",
    ax=ax
)

# This code block is for plotting each line segment with a differnt color.
pts = np.vstack([means["rural_mean"].values, means["suhii_mean"].values]).T 
pts_offset = np.roll(pts, 1, axis=0)
segments = np.array([np.vstack([pts[i,:], pts_offset[i,:]]) for i in range(13)])
dydx = np.linspace(0, 1, 11)
norm = plt.Normalize(dydx.min(), dydx.max())
cmap = sns.color_palette("ch:s=-.2,r=.6", as_cmap=True)
lc = LineCollection(segments, cmap=cmap, norm=norm, zorder=2)
lc.set_array(dydx)
lc.set_linewidth(5)
line = ax.add_collection(lc)

# Plot the monthly values
ax.scatter(
    means["rural_mean"],
    means["suhii_mean"],
    color="black",
    s=25,
    marker="o",
    zorder=3,
)

ax.axhline(y=0, color="black", ls="dashed")

ax.set_xlabel("Rural LST [K]")
ax.set_ylabel("SUHII [K]")

## Task 5

For this final task we will the monthly means I pre-calculates from our DB for almost 1500 cities around the globe.

Our goal is to visualize the SUHII Seasonal hysteresis for all the densely-populated climate zones.

In [ ]:
# Let's start by making a function to retrieve these data:

def get_monthy_means(climate: str):

    filename = os.path.join(LST_DIR, "SUHII_monthly_means.db")
    connection = sqlite3.connect(filename)

    query = """
    SELECT data_day.UID, months_relative_to_SE, suhii_mean, rural_mean
    FROM data_day 
    LEFT JOIN cities
    ON data_day.UID=cities.UID
    WHERE cities.climate=:climate_zone
    """

    query_args = {
        "climate_zone": climate,
    }

    data = pd.read_sql(
        sql = query,
        con = connection,
        params = query_args,
    )
    
    connection.close()

    return data

In [ ]:
# Let's visialize the data

ZONES = [
    "Aw",
    "BSh",
    "BSk",
    "Csa",
    "Cwa",
    "Cwb",
    "Cfa",
    "Cfb",
    "Dwa",
    "Dfa",
    "Dfb",
]

fig, axs = plt.subplots(ncols=4, nrows=3, figsize=(6.0, 4.5), sharex=True, sharey=True)

axs = axs.flatten()
axs[-1].remove()

for i, (zone, ax) in enumerate(zip(ZONES, axs)):

    loops_indiv_cities = get_monthy_means(zone)

    for UID in loops_indiv_cities["UID"].unique():

        city_mask = loops_indiv_cities["UID"] == UID
        city_loop = loops_indiv_cities.loc[city_mask]
        city_loop = city_loop.append(city_loop.iloc[0])  # close the loop

        ax.plot(
            city_loop["rural_mean"],
            city_loop["suhii_mean"],
            lw=1,
            color=np.random.rand(3,), # color each loop with a random color
            alpha = 0.35,
            zorder=1,
        )

    ax.tick_params(labelsize=6, which="both", width=0.5, length=2)
    ax.set_title(f"{zone}", fontsize=7)

    if i in [8, 9, 10]:
        ax.set_xlabel("Rural LST [K]", fontsize=7)

    if i in [0, 4, 5]:
        ax.set_ylabel("SUHII [K]", fontsize=7)

    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.7)
        
fig.tight_layout()
plt.show()

**Well done!**

The results that you see here have been published in: https://doi.org/10.3390/rs14102318
